In [1]:
from keras.models import load_model  # TensorFlow is required for Keras to work
import cv2  # Install opencv-python
import numpy as np
import requests
from bs4 import BeautifulSoup
from PIL import ImageFont, ImageDraw, Image    # 載入 PIL 相關函式庫
import time

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = [label.strip() for label in open("labels.txt", "r", encoding="utf-8").readlines()]

# CAMERA can be 0 or 1 based on default camera of your computer
camera = cv2.VideoCapture(0)

image_resized = None

# Create a window to display the results
cv2.namedWindow("Webcam Image with Text", cv2.WINDOW_NORMAL)

urls = ["https://online.carrefour.com.tw/zh/1402001600101.html",
        "https://online.carrefour.com.tw/zh/1505001700101.html",
        "https://online.carrefour.com.tw/zh/gery/1413107400101.html",
        "https://online.carrefour.com.tw/zh/1501202200106.html",
        "https://online.carrefour.com.tw/zh/%E7%B5%B1%E4%B8%80/1450300300103.html"]

while True:
    # Grab the webcamera's image.
    ret, image = camera.read()

    # Resize the raw image into (224-height,224-width) pixels
    image_resized = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

    # Show the image in a window
    #cv2.imshow("Webcam Image", image_resized)

    # Add a delay to keep the window open for a short time
    #cv2.waitKey(1000) 

    # Make the image a numpy array and reshape it to the models input shape.
    image_input = np.asarray(image_resized, dtype=np.float32).reshape(1, 224, 224, 3)

    # Normalize the image array
    image_input = (image_input / 127.5) - 1

    # Predicts the model
    prediction = model.predict(image_input)
    index = np.argmax(prediction)
    class_name = class_names[index]
    class_name = class_name[2:]
    confidence_score = prediction[0][index]

    for url in urls:
        # 發送 GET 請求
        response = requests.get(url)

        #查詢時間
        local_time = time.ctime(time.time())
        
        # 檢查請求是否成功
        if response.status_code == 200:
            # 使用 BeautifulSoup 解析 HTML
            soup = BeautifulSoup(response.text, 'html.parser')
        
            # 找到包含商品信息的 div 元素
            goods_info_div = soup.find('div', class_='goods-info')
        
            # 檢查是否找到了商品信息元素
            if goods_info_div:
                # 找到商品名稱
                product_name = goods_info_div.find('h1').text.strip()
        
                if product_name == class_name:
                    # 找到商品價格
                    product_price = goods_info_div.find('span', class_='money').text.strip()

                    # 在图像上绘制检测结果
                    fontpath = "C:/Users/USER/pythonwork/final/NotoSansTC-VariableFont_wght.ttf"
                    font_size = 10
                    # 使用try-except塊處理錯誤
                    try:
                        font = ImageFont.truetype(fontpath, font_size)
                    except IOError:
                        print(f"Error: Cannot load font from {fontpath}")
                        font = ImageFont.load_default()  # 使用默認字型

                    image_resized = image_resized.astype(np.uint8)
                    imgPil = Image.fromarray(image_resized)   # Convert the image to a PIL image
                    draw = ImageDraw.Draw(imgPil)                # 準備開始畫畫

                    # 添加文字背景
                    rect_height = 70
                    draw.rectangle([(0, 150), (imgPil.width, 150 + rect_height)], fill=(255, 255, 255))
                    # 應用粗體效果
                    draw.text((0, 150), f'商品名稱: {product_name}\n商品價格: ${product_price}\n相似程度: {np.round(confidence_score * 100)}%\n查詢時間: {local_time}'
                              , fill=(0, 0, 0), font=font)
                    draw.text((0, 150), f'商品名稱: {product_name}\n商品價格: ${product_price}\n相似程度: {np.round(confidence_score * 100)}%\n查詢時間: {local_time}'
                              , fill=(0, 0, 0), font=font)
                    
                    img = np.array(imgPil)
                    cv2.imshow("Webcam Image with Text", img)  # Show the image with text
                
                    print("商品名稱:", product_name)
                    print("商品價格:", product_price)
            else:
                print("未找到商品信息元素")
        else:
            print("請求失敗")

    
    # Print prediction and confidence score
    print("相似程度:", str(np.round(confidence_score * 100))[:-2], "%")

    print("查詢時間:",local_time)

    # Listen to the keyboard for presses.
    keyboard_input = cv2.waitKey(1)

    # 27 is the ASCII for the esc key on your keyboard.
    if keyboard_input == 27:
        break

camera.release()
cv2.destroyAllWindows()

1/1 [==============================] - 1s 613ms/step
商品名稱: Gery捲心酥(黑巧克力味)
商品價格: 42
相似程度: 100 %
查詢時間: Wed Dec 20 03:23:17 2023
1/1 [==============================] - 0s 29ms/step
商品名稱: Gery捲心酥(黑巧克力味)
商品價格: 42
相似程度: 100 %
查詢時間: Wed Dec 20 03:23:22 2023
1/1 [==============================] - 0s 28ms/step
商品名稱: Gery捲心酥(黑巧克力味)
商品價格: 42
相似程度: 100 %
查詢時間: Wed Dec 20 03:23:30 2023
1/1 [==============================] - 0s 27ms/step
商品名稱: 光泉果汁時刻 100%柳橙綜合果汁-290ml到貨效期約6-8天
商品價格: 32
相似程度: 92 %
查詢時間: Wed Dec 20 03:23:35 2023
1/1 [==============================] - 0s 26ms/step
商品名稱: 光泉果汁時刻 100%柳橙綜合果汁-290ml到貨效期約6-8天
商品價格: 32
相似程度: 81 %
查詢時間: Wed Dec 20 03:23:40 2023
1/1 [==============================] - 0s 27ms/step
商品名稱: 光泉果汁時刻 100%柳橙綜合果汁-290ml到貨效期約6-8天
商品價格: 32
相似程度: 52 %
查詢時間: Wed Dec 20 03:23:45 2023
1/1 [==============================] - 0s 28ms/step
商品名稱: Gery捲心酥(黑巧克力味)
商品價格: 42
相似程度: 54 %
查詢時間: Wed Dec 20 03:23:50 2023
1/1 [==============================] - 0s 26ms/step
商品名稱: Gery捲心酥(黑巧克力味)
商